In [3]:
# !pip install sqlalchemy
!pip install pymysql

  Using cached PyMySQL-1.1.0-py3-none-any.whl.metadata (4.4 kB)
Using cached PyMySQL-1.1.0-py3-none-any.whl (44 kB)


In [4]:
import pandas as pd
import json
import numpy as np
from sqlalchemy import create_engine
import pymysql
import os

In [6]:
def uploadToMariaDB(table, df, conn):
  if not isinstance(df, pd.DataFrame):
    raise Exception("Not a dataframe")
  remote_db_table = pd.DataFrame()
  try:
    remote_db_table = pd.read_sql("Select * from {table} limit 10;".format(table=table), conn)
  except Exception as  e:
    print(e)
    remote_db_table["id"] = np.NaN

  incidence = ~df["id"].isin(remote_db_table["id"])
  response_frame = df[incidence]
  if df.shape[0] > 0:
    rows = df.to_sql(table, con=conn, if_exists='append', index = False)
    print(f"Records uploaded {rows}")
  else:
    print("Nothing to load")

In [8]:
database_user = os.getenv("database_user")
database_password = os.getenv("database_password")
database_host = os.getenv("database_host")
database = os.getenv("database")

In [7]:
connString = "mysql+pymysql://{user}:{password}@{host}:3306/{database}".format(user=database_user, password=database_password, host=database_host, database=database)
sqlEngine = create_engine(connString, pool_recycle=3600)

In [10]:
df = pd.read_csv('../files/QQP_2015/csv/012015.csv')
df.columns = ['PRODUCT', 'PRESENTATION', 'BRAND', 'CATEGORY', 'CATALOG', 'PRICE', 'REGISTRATION_DATE', 'COMMERCIAL_CHAIN', 'BUSINESS_LINE',
              'COMMERCIAL_NAME', 'ADDRESS', 'STATE', 'MUNICIPALITY', 'LATITUDE', 'LONGITUDE']
df.head()

,PRODUCT,PRESENTATION,BRAND,CATEGORY,CATALOG,PRICE,REGISTRATION_DATE,COMMERCIAL_CHAIN,BUSINESS_LINE,COMMERCIAL_NAME,ADDRESS,STATE,MUNICIPALITY,LATITUDE,LONGITUDE
0,BATIDORAS,2532 Ó 2532-013. MANUAL. 6 VELOCIDADES. SIN TAZON,OSTER,APARATOS ELECTRICOS,ELECTRODOMESTICOS,441.15,2015-01-02,LIVERPOOL,TIENDA DEPARTAMENTALES,LIVERPOOL SUCURSAL AGUASCALIENTES,"ZACATECAS NTE. 851, ESQ. ALTARIA, COL. TROJES ...",AGUASCALIENTES,AGUASCALIENTES,21.924271,-102.289721
1,BATIDORAS,2614 O 2614-13. DE INMERSION. 2 VELOCIDADES. S...,OSTER,APARATOS ELECTRICOS,ELECTRODOMESTICOS,424.15,2015-01-02,LIVERPOOL,TIENDA DEPARTAMENTALES,LIVERPOOL SUCURSAL AGUASCALIENTES,"ZACATECAS NTE. 851, ESQ. ALTARIA, COL. TROJES ...",AGUASCALIENTES,AGUASCALIENTES,21.924271,-102.289721
2,BATIDORAS,2700 O 2700-013. PEDESTAL. 12 VELOCIDADES. TAZ...,OSTER,APARATOS ELECTRICOS,ELECTRODOMESTICOS,2591.65,2015-01-02,LIVERPOOL,TIENDA DEPARTAMENTALES,LIVERPOOL SUCURSAL AGUASCALIENTES,"ZACATECAS NTE. 851, ESQ. ALTARIA, COL. TROJES ...",AGUASCALIENTES,AGUASCALIENTES,21.924271,-102.289721
3,BATIDORAS,FPSTHS2611 Ó FPSTHS2611-013. MANUAL Y PEDESTAL...,OSTER,APARATOS ELECTRICOS,ELECTRODOMESTICOS,721.65,2015-01-02,LIVERPOOL,TIENDA DEPARTAMENTALES,LIVERPOOL SUCURSAL AGUASCALIENTES,"ZACATECAS NTE. 851, ESQ. ALTARIA, COL. TROJES ...",AGUASCALIENTES,AGUASCALIENTES,21.924271,-102.289721
4,CAFETERAS,FG4105ME. CAPACIDAD 10 TAZAS,MOULINEX. HELIORA SELECTIVE,APARATOS ELECTRICOS,ELECTRODOMESTICOS,559.20,2015-01-02,LIVERPOOL,TIENDA DEPARTAMENTALES,LIVERPOOL SUCURSAL AGUASCALIENTES,"ZACATECAS NTE. 851, ESQ. ALTARIA, COL. TROJES ...",AGUASCALIENTES,AGUASCALIENTES,21.924271,-102.289721


In [11]:
with sqlEngine.connect() as conn:
  uploadToMariaDB('QQP_DETAIL', df, conn)
  conn.commit()

KeyError: 'id'